In [100]:
import pandas as pd
import json
df = pd.read_csv('24_train_1.csv')

In [101]:
df

,ArticleId,Text,Category
0,1429,sfa awaits report over mikoliunas the scottish...,sport
1,1896,parmalat to return to stockmarket parmalat th...,business
2,1633,edu blasts arsenal arsenal s brazilian midfiel...,sport
3,2178,henman decides to quit davis cup tim henman ha...,sport
4,194,french suitor holds lse meeting european stock...,business
...,...,...,...
995,1250,blair damaged by blunkett row a majority of ...,politics
996,1639,a november to remember last saturday one news...,sport
997,916,highbury tunnel players in clear the football ...,sport
998,2217,top stars join us tsunami tv show brad pitt r...,entertainment


In [102]:
import string
import nltk
# import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import *

# stemming tool from nltk
stemmer = PorterStemmer()
# a mapping dictionary that help remove punctuations
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)
def get_tokens(text):
  # turn document into lowercase
  lowers = text.lower()
  # remove punctuations
  no_punctuation = lowers.translate(remove_punctuation_map)
  # tokenize document
  tokens = nltk.word_tokenize(no_punctuation)
  # remove stop words
  filtered = [w for w in tokens if not w in stopwords.words('english')]
  # stemming process
  stemmed = []
  for item in filtered:
      stemmed.append(stemmer.stem(item))
  # final unigrams
  return stemmed

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/gayathrisjs/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gayathrisjs/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [103]:
df['tokens']=df['Text'].apply(lambda row: get_tokens(row))
#df1=df['Text'].apply(lambda row: get_tokens(row.to_string(index=False)), axis=1)
# df['tokens']=df.apply(lambda x: get_tokens(''.join(x)),axis=1)
# df[['ArticleId', 'Text', 'Category']] = df.apply(lambda row: row.apply(get_tokens), axis=1)


In [104]:
df

,ArticleId,Text,Category,tokens
0,1429,sfa awaits report over mikoliunas the scottish...,sport,"[sfa, await, report, mikoliuna, scottish, foot..."
1,1896,parmalat to return to stockmarket parmalat th...,business,"[parmalat, return, stockmarket, parmalat, ital..."
2,1633,edu blasts arsenal arsenal s brazilian midfiel...,sport,"[edu, blast, arsen, arsen, brazilian, midfield..."
3,2178,henman decides to quit davis cup tim henman ha...,sport,"[henman, decid, quit, davi, cup, tim, henman, ..."
4,194,french suitor holds lse meeting european stock...,business,"[french, suitor, hold, lse, meet, european, st..."
...,...,...,...,...
995,1250,blair damaged by blunkett row a majority of ...,politics,"[blair, damag, blunkett, row, major, voter, 68..."
996,1639,a november to remember last saturday one news...,sport,"[novemb, rememb, last, saturday, one, newspap,..."
997,916,highbury tunnel players in clear the football ...,sport,"[highburi, tunnel, player, clear, footbal, ass..."
998,2217,top stars join us tsunami tv show brad pitt r...,entertainment,"[top, star, join, us, tsunami, tv, show, brad,..."


In [105]:
with open('dictionary.txt','r') as dicti:
    dictionary= set(dicti.read().splitlines())
with open('dictionary.txt','r') as dicti:
    dictionary_list= list(dicti.read().splitlines())

In [106]:
# len(dictionary)
print(dictionary_list)

['000', '1', '10', '100', '11', '12', '13', '14', '15', '16', '18', '2', '20', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '24', '25', '3', '30', '4', '40', '5', '50', 'abil', 'abl', 'accept', 'access', 'accord', 'account', 'accus', 'achiev', 'across', 'act', 'action', 'activ', 'actor', 'actress', 'ad', 'add', 'address', 'admit', 'affect', 'africa', 'age', 'agenc', 'ago', 'agre', 'agreement', 'ahead', 'aid', 'aim', 'air', 'airlin', 'album', 'alleg', 'allow', 'almost', 'alreadi', 'also', 'although', 'alway', 'america', 'american', 'among', 'amount', 'analyst', 'announc', 'annual', 'anoth', 'answer', 'anyth', 'appeal', 'appear', 'appl', 'approach', 'approv', 'april', 'area', 'argu', 'around', 'arrest', 'arsen', 'artist', 'ask', 'associ', 'asylum', 'athlet', 'attack', 'attempt', 'attend', 'attract', 'audienc', 'australia', 'australian', 'author', 'avail', 'averag', 'aviat', 'avoid', 'award', 'away', 'back', 'bad', 'ball', 'ban', 'band', 'bank', 'bankruptci', 'base', 'battl', '

In [107]:
df.columns

Index(['ArticleId', 'Text', 'Category', 'tokens'], dtype='object')

In [108]:
df['filtered tokens']= df['tokens'].apply(lambda row: [text for text in row if text in dictionary])

In [109]:
import numpy as np

def tfidf_matrix(rows, dictionary_list):
    n=len(rows)
    m=len(dictionary)
    
    tfidf=np.zeros((n,m))
    idf_j={}
    
    
    
    # word_doc_frequency= {dic: sum(1 for row)}
    for dic in dictionary_list:
        count= rows.apply(lambda row: 1 if dic in row else 0)
        total_doc_count= count.sum()
        idf_j[dic]= np.log(n/total_doc_count)
    
    print(count.sum(),len(count))
    
    for i,row in enumerate(rows):
        #print('row=',row)
        frequency_word= {w: row.count(w) for w in row}
        frequency_max= max(frequency_word.values())
        # print(i, frequency_max,frequency_word)
        for j,dic in enumerate(dictionary_list):
            tf_ij=frequency_word.get(dic,0)/frequency_max
            tfidf[i,j]=tf_ij *idf_j[dic]
    return tfidf,idf_j
    

In [110]:
tfidf,idf_j=tfidf_matrix(df['filtered tokens'],dictionary_list)

21 1000


In [111]:
tfidf

array([[0.28878469, 0.        , 0.36036196, ..., 0.51540439, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.15158953, 0.        ,
        0.        ]])

In [112]:
filename='matrix.txt'
np.savetxt(filename, tfidf, fmt='%f', delimiter=',')

In [113]:
categories = df['Category'].unique()
categories

array(['sport', 'business', 'tech', 'entertainment', 'politics'],
      dtype=object)

In [114]:
count={}
for category in categories:
    count[category] = {}
    category_df = df[df['Category'] == category]
    # break
    summ=0
    for word in dictionary:
        summ=0
        for row in category_df['filtered tokens']:
            summ+= row.count(word)
        count[category][word]=summ
    sorted_list = sorted(count[category].items(), key=lambda item: -item[1])
    sorted_list = sorted_list[:3]
    count[category] = dict(sorted_list)
print(count)
with open("frequency.json", "w") as outfile:
    json.dump(count, outfile)
        
        # count[category][word]=category_df['Text'].apply(lambda row: sum(1 if word in row else 0))
        


{'sport': {'said': 428, 'game': 353, 'win': 288}, 'business': {'said': 724, 'us': 377, 'year': 360}, 'tech': {'said': 757, 'use': 459, 'peopl': 427}, 'entertainment': {'film': 450, 'said': 386, 'year': 249}, 'politics': {'said': 996, 'mr': 726, 'would': 495}}


In [115]:
count

{'sport': {'said': 428, 'game': 353, 'win': 288},
 'business': {'said': 724, 'us': 377, 'year': 360},
 'tech': {'said': 757, 'use': 459, 'peopl': 427},
 'entertainment': {'film': 450, 'said': 386, 'year': 249},
 'politics': {'said': 996, 'mr': 726, 'would': 495}}

In [116]:
category_dic={}
for category in categories:
    category_dic[category]={}
    category_loc= df[df['Category']==category].index
    #print(category,category_loc)
    category_matrix=tfidf[category_loc]
    #print(category,category_matrix)
    req_average=np.mean(category_matrix,axis=0)
    # top_ind= np.argsort(req_average)[-3: ][::-1]
    mapped_list = []
    for index, value in enumerate(req_average):
        mapped_list.append((value, index))
    mapped_list.sort(key = lambda m : m[0], reverse = True)
    mapped_lis = mapped_list[:3]
    for ind in mapped_lis:
        category_dic[category][dictionary_list[ind[1]]]= ind[0]
with open("score.json", "w") as outfile:
    json.dump(category_dic, outfile)
    
        

In [117]:
mapped_lis

[(0.45105036714182084, 507),
 (0.4313731783204545, 312),
 (0.42043597469422206, 601)]

In [118]:
category_dic

{'sport': {'game': 0.35727414648708805,
  'england': 0.31907434737608514,
  'win': 0.30741067997068455},
 'business': {'firm': 0.2891252868078186,
  'bank': 0.2697288199539767,
  'market': 0.2616290834155383},
 'tech': {'mobil': 0.3462714837303001,
  'phone': 0.3319065027131584,
  'softwar': 0.3152238172837377},
 'entertainment': {'film': 0.7216412939111394,
  'award': 0.4106447057087541,
  'star': 0.40803563438879187},
 'politics': {'labour': 0.45105036714182084,
  'elect': 0.4313731783204545,
  'mr': 0.42043597469422206}}

In [119]:
dictionary_list[100]

'avail'

In [120]:
category_matrix

array([[0.30941217, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.86635408, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 1.71801463, 0.        ,
        0.        ],
       ...,
       [0.20627478, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.22214207, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.21810799, 0.        , ..., 0.        , 0.        ,
        0.        ]])